# **Proyek Sistem Rekomendasi: Rekomendasi Film Tahun 1996 - 2018**

## **Import Library**

In [3]:
# Melakukan import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Insight:**

- Pandas: untuk manipulasi dan analisis data
- Numpy: untuk komputasi numerik
- Seaborn: untuk visualisasi data statistik
- Matplotlib: untuk visualisasi data

## **Loading Dataset**

**Melakukan loading dataset menggunakan pandas**

In [5]:
links_df = pd.read_csv("data/links.csv")
movies_df = pd.read_csv("data/movies.csv")
ratings_df = pd.read_csv("data/ratings.csv")
tags_df = pd.read_csv("data/tags.csv")

In [10]:
links_df.head(5)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [11]:
movies_df.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
ratings_df.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [13]:
tags_df.head(5)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


**Insight:**

- Pada proyek ini terdapat empat dataset yaitu links, movies, ratings dan tags
- Dataset ratings berisi baris data yang mewakili satu peringkat, satu movie oleh satu pengguna. Kolom pada dataset ini adalah userId, movieId, rating dan timestamp
- Dataset links berisi data movie dengan tautan ke sumber data film yaitu IMDB dan TMDB. Kolom pada dataset ini adalah movieId, imdbId dan tmdbId
- Dataset movies berisi informasi dari film yaitu judul dan genre. Kolom pada dataset ini adalah movieId, title dan genres
- Dataset tags berisi informasi tag yang diberikan oleh satu user pada satu film. Kolom pada dataset ini adalah userId, movieId, tag dan timestamp


## **Exploratory Data Analysis (EDA)**

### **Melihat Informasi Dataset**

In [19]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


**Insight:**

Berdasarkan output di atas, dataset movies memiliki 7742 entri. Terdapat tiga variabel pada dataset ini yaitu movieId, title atau judul film dan genres.


In [21]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


**Insight:**

Berdasarkan output di atas, dataset ratings memiliki 100835 entri. Terdapat empat variabel pada dataset ini yaitu userId, movieId, rating dan timestamp.

In [22]:
links_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB


**Insight:**

Berdasarkan output di atas, dataset links memiliki 9741 entri. Terdapat tiga variabel pada dataset ini yaitu movieId, imdbId dan tmdbId.

In [23]:
tags_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


**Insight:**

Berdasarkan output di atasm dataset tags memiliki 3682 entri. Terdapat empat variabel pada dataset ini yaitu userId, movieId, tag dan timestamp.

In [39]:
print('Jumlah userId: ', len(ratings_df['userId'].unique()))
print('Jumlah data film: ', len(movies_df['movieId']))
print('Jumlah data rating: ', len(ratings_df['rating']))

Jumlah userId:  610
Jumlah data film:  9742
Jumlah data rating:  100836


**Insight:**

Terdapat 610 user yang memberikan rating pada 9742 film. Kemudian untuk akumulasi rating yang diberikan user totalnya adalah 100836.

### **Mengecek Missing Value**

In [24]:
links_df.isna().sum()

movieId    0
imdbId     0
tmdbId     8
dtype: int64

In [25]:
ratings_df.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [33]:
movies_df.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [27]:
tags_df.isna().sum()

userId       0
movieId      0
tag          0
timestamp    0
dtype: int64

**Insight:**

Dari keempat dataset di atas, setelah mengecek menggunakan isna() tidak ditemukan adanya missing value/Nan.

### **Melihat Deskripsi Statistik pada Dataset Rating**

In [34]:
ratings_df.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


**Insight:**

Dari output di atas, ditemukan bahwa nilai minimum user memberikan rating adalah 0.5 dan maksimum ratingnya adalah 5. Ini artinya, skala rating berkisar antara 0.5 hingga 5. Kemudian untuk rata-rata user memberikan rating 3.5.

### **Mengecek Nilai Duplikat pada Dataset**

In [31]:
print("Jumlah data duplikat pada links.csv: ", links_df.duplicated().sum())
print("Jumlah data duplikat pada movies.csv: ", movies_df.duplicated().sum())
print("Jumlah data duplikat pada ratings.csv: ", ratings_df.duplicated().sum())
print("Jumlah data duplikat pada tags.csv: ", tags_df.duplicated().sum())

Jumlah data duplikat pada links.csv:  0
Jumlah data duplikat pada movies.csv:  0
Jumlah data duplikat pada ratings.csv:  0
Jumlah data duplikat pada tags.csv:  0


**Insight:**

Dari keempat dataset di atas, setelah melakukan pengecekan dengan fungsi duplicated() tidak ditemukan adanya data yang sama/duplikat.

## **Data Preprocessing**

## **Data Preparation**

## **Building Model**

## **Evaluating Model**

## **Hyperparameter Tuning**

## **Evaluating Model after Tuning**